<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from scipy.sparse import lil_matrix
import edward as ed
from edward.models import Beta
import tensorflow as tf
import numpy as np
import math

//anaconda/envs/py35tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [ ]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = -1
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

In [ ]:
np_cascades = np.zeros((len(v),numNodes))
for row, cascade in enumerate(v):  
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for col in range(len(c_nodes)):
        np_cascades[row][c_nodes[col]]=c_times[col]

In [ ]:
def infectedCascade(cascade, start,N, T=10):
    A_inf = lil_matrix((cascade.shape[0],cascade.shape[0]))

    infected = np.nonzero(cascade)[0]
    c_nodes= cascade.argsort()[-len(infected):]

    for i in range(len(c_nodes)):
        for j in range(i):
            if cascade[j] < T:
                A_inf[(c_nodes[i],c_nodes[j])]=cascade[i]-cascade[j]
    
    return A_inf

In [ ]:
def uninfectedCascade(cascade,start,N,T=10):
    nodes = {s for s in range(N)}
    A_uninf = lil_matrix((N,N))

    infected = np.nonzero(cascade)[0]
    c_nodes= cascade.argsort()[-len(infected):]

    for i in range(len(c_nodes)):
        for j in (nodes-set(c_nodes)):
            A_uninf[c_nodes[i],j]=T-cascade[i]

    return A_uninf

In [ ]:
def sigmoid(x):
    try: 
        return 1 / (1 + math.exp(-x))
    except OverflowError:
        if x > 0: 
            return 1
        else: 
            return 0


In [ ]:
numNodes = 5
#np_cascades.shape


In [ ]:
A = Beta(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))
cascade_ph = Cascade(A)

sess.run(cascade_ph._log_prob(np.array([0,1,1,0,0])))

# dir(cascade_ph)

In [ ]:
np_cascades.shape

In [ ]:
data = {cascade_ph: np_cascades[0]}
inference = ed.MAP([A], data)

In [ ]:
sess = ed.get_session()
tf.global_variables_initializer().run()

inference.run(n_iter=2500)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.framework import tensor_shape

from edward.models.random_variable import RandomVariable
from tensorflow.contrib.distributions import Distribution

try:
    from tensorflow.contrib.distributions import FULLY_REPARAMETERIZED
except Exception as e:
    raise ImportError("{0}. Your TensorFlow version is not supported.".format(e))

class Cascade(RandomVariable, Distribution):
    def __init__(self,
                params,
                seed_set = None,
                seed_set_param = 0.01,
                validate_args=False,
                allow_nan_stats=True,
                name="Cascade"):
        """Initialize a `Cascade` random variable.
        Args:
            params: tf.Tensor.
            Collection of samples. Its outer (left-most) dimension
            determines the number of samples.
        """
        parameters = locals()
        with tf.name_scope(name, values=[params]):
            with tf.control_dependencies([]):
                self._params = tf.identity(params, name="alpha")
                self._seed_set = seed_set
                self._seed_set_param = seed_set_param
                try:
                    self._n = tf.shape(self._params)[0] 
                except ValueError:  # scalar params
                    self._n = tf.constant(1)
                    
        super(Cascade, self).__init__(
            dtype=self._params.dtype,
            reparameterization_type=FULLY_REPARAMETERIZED,
            validate_args=validate_args,
            allow_nan_stats=allow_nan_stats,
            parameters=parameters,
            graph_parents=[self._params, self._n],
            name=name)
    
    @staticmethod
    def _param_shapes(sample_shape):
        return {"params": tf.convert_to_tensor(sample_shape, dtype=tf.int32)}

    @property
    def params(self):
        """Distribution parameter."""
        return self._params

    @property
    def n(self):
        """Number of samples."""
        return self._n

    def _batch_shape_tensor(self):
        return array_ops.shape(self.params)

    def _batch_shape(self):
        return self.params.shape[1:]

    def _event_shape_tensor(self):
        return tf.shape(self.params)[1:]

    def _event_shape(self):
        return tensor_shape.scalar()


    
    def _log_prob(self, value):

        #alpha_tensor = tf.convert_to_tensor(tf.cast(self.params,dtype=tf.float64),dtype=tf.float64)
        alpha_tensor = tf.Variable(self.params, dtype=tf.float64)
        
        I = tf.convert_to_tensor(infectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)
        U = tf.convert_to_tensor(uninfectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)


        psi_1 = -tf.reduce_sum(tf.multiply(alpha_tensor,I))
        psi_2 = -tf.reduce_sum(tf.multiply(tf.transpose(alpha_tensor),U))
        
        zero_pos = tf.add(tf.multiply(tf.sign(I),-1),tf.ones((self.params.shape[0],self.params.shape[0]),dtype=tf.float64))
        psi_3 = tf.reduce_sum(tf.log(tf.add(zero_pos,tf.multiply(tf.transpose(alpha_tensor),tf.sign(I)))))

        return psi_1 + psi_2 + psi_3

    def _sample_n(self, n=1, seed=None):

        # generate the cascades
        #if self._seed_set: 
        #    seedset = self._seed_set
        #else: 
        #    seedset = np.random.binomial(1, self._seed_set_param,size=[N,1])
        
        #I_t = tf.convert_to_tensor(seedset)
        #I_t_1 = tf.zeros((N,1))
        A = self.parameters['params']
        N = A.shape[0]
        
        #seedset = np.random.randint(2, size=N)
        seedset = np.random.binomial(1, 0.0067,size=[N,1])
        data = np.zeros((n,N))
        for s in range(n):
            I_t = np.array(seedset).reshape(N,1)
            I_t_1 = np.zeros((N,1))
            #beta_tau = np.matmul(beta, tau)
            i = 1
            data1 = np.zeros((N,1))
            while not(np.array_equal(I_t,I_t_1)): 
                I_new = I_t - I_t_1
                data1 += I_new*i
                I_uninf = np.ones((N,1)) - I_t 
                AI = np.matmul(A.T, I_new)
                # calculate probability
                p = AI
                p = np.apply_along_axis(sigmoid,1, p).reshape(N,1) * I_uninf

                # draw random vector
                I_t_1 = I_t
                I_t = I_t_1 + np.random.binomial(1, p)
                i += 1
            data[s] = data1.flatten()
            
        
        return tf.convert_to_tensor(data.reshape((n,N)))
        """
                

        
        input_tensor = self.params
        if len(input_tensor.shape) == 0:
            input_tensor = tf.expand_dims(input_tensor, 0)
            multiples = tf.concat([tf.expand_dims(n, 0), [1] * len(self.event_shape)], 0)
            return tf.tile(input_tensor, multiples)
        else: 
            probs = tf.ones([self.n]) / tf.cast(self.n, dtype=tf.float32)
            cat = tf.contrib.distributions.Categorical(probs)
            indices = cat._sample_n(n, seed)
            print(input_tensor.shape)
            print(indices.shape)
            tensor = tf.gather(input_tensor, indices)
            print(tensor.shape)
            return tensor
        """

In [287]:
## Initialize the values 
# Cascades should be of size c x n; Initialize the variables
# TO DO: make A only NxN not c x NxN !!!!
C = tf.placeholder(tf.float32, [c,N])
A = Gamma(concentration=tf.zeros((c,N,N)), rate=tf.ones((c,N,N)))

I_inf = tf.expand_dims(tf.where(C>0, x=tf.ones((c,N)), y=tf.zeros((c,N))),1)
I_uninf = 1 - tf.expand_dims(tf.where(C>0, x=tf.ones((c,N)), y=tf.zeros((c,N))),1)
t_max = tf.expand_dims(tf.reduce_max(C,1),1)

#calculate dist1
t_dist1 = tf.expand_dims(tf.subtract(t_max,C),1)

#calculate dist2
k = tf.concat([-tf.ones((c,N,1)),tf.expand_dims(C,-1)], 2)
k_T = tf.transpose(tf.concat([tf.expand_dims(C,-1),tf.ones((c,N,1))], 2),perm=[0,2,1])
t_dist2 = tf.matmul(k,k_T)

# calculate psi_1 

psi_1 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_uninf, perm=[0,2,1]))
psi_1 = tf.multiply(psi_1,t_dist1)
psi_1 = tf.reduce_sum(-psi_1)

# calculate psi_2

psi_2 = tf.multiply(tf.where(t_dist2 > 0, A, tf.zeros((c,N,N))),tf.transpose(I_inf, perm=[0,2,1]))
psi_2 = tf.multiply(psi_2,t_dist2)
psi_2 = tf.reduce_sum(-psi_2)

# calculate psi_3

psi_3 = tf.multiply(tf.where(t_dist2 > 0, A, tf.zeros((c,N,N))), I_inf)
psi_3 = tf.transpose(tf.log(tf.reduce_sum(psi_3,axis=2,keepdims=True)), perm=[0,2,1])
psi_3 = tf.where(tf.logical_not(tf.is_inf(psi_3)), psi_3 , tf.zeros((c,1,N)))
psi_3 = tf.reduce_sum(-psi_3)

# log(p)

log_p = psi_1 + psi_2
c_i = tf.exp(log_p)

I_uninf = tf.ones((c,N)) - I_inf

# define log(p(c,A)) for a single cascade
data={C: [[0,0,1,2,3],[0,1,0,2,3]]}
sess = tf.Session()
sess.run(log_p, feed_dict=data)

TypeError: Error converting shape to a TensorShape: int() argument must be a string, a bytes-like object or a number, not 'Tensor'.

In [ ]:
alpha_test = np.random.uniform(0,1,[5,5])
# print(v[0])
x = Cascade(alpha_test)
sess = tf.InteractiveSession()
sess.run(x.sample(100))

In [ ]:
N = 100
D = 10

X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

qw = Normal(loc=tf.Variable,
            scale=tf.nn.softplus(tf.get_variable("qw/scale3", [D])))
qb = Normal(loc=tf.get_variable("qb/loc3", [1]),
            scale=tf.nn.softplus(tf.get_variable("qb/scale3", [1])))

In [ ]:
X_train = np.random.binomial(1, 0.01,size=[N,D])
y_train = np.random.binomial(1, 0.01,size=[N])

inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(n_samples=5, n_iter=250)

In [ ]:
Aq = tf.Variable(tf.float32, [numNodes, numNodes])

In [ ]:
from edward.models import PointMass
A = Beta(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))
cascade = Cascade(params=A)


In [ ]:
qA = PointMass(params= tf.get_variable("qA2", [numNodes,numNodes], dtype=tf.float32))

In [ ]:
qA = tf.get_variable("qA1", [numNodes,numNodes])

In [ ]:
cascade

In [ ]:
# simulate data 
X_train = np.random.binomial(1, 0.01,size=[N,numNodes])

#model
prior = Beta(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))
cascade=Cascade(params=prior)

#run inference
A = ed.models.PointMass(params=tf.Variable(tf.zeros([numNodes, numNodes])))
inference = ed.MAP({cascade: A}, data={X: X_train[0]})
inference.run()

In [ ]:
numNodes = 5
data = np.random.binomial(1, 0.01,size=[100,numNodes])
A = ed.models.PointMass(params=tf.Variable(tf.zeros([numNodes,numNodes])))

inference = ed.MAP({params: A}, data = {cascade: data})

In [ ]:
s = np.array([[1,2],[1,1]])
s2 = np.array([0,0])

In [ ]:
x.shape

In [ ]:
s

In [ ]:
alpha_test = np.ones((3,3),dtype=np.float64)/2

testcascade = Cascade(alpha_test)

sess = tf.InteractiveSession()
test_list = [1., 0., 0., 1.5]
sess.run(testcascade._log_prob(test_list))

In [ ]:
sess.run(cascade_ph._log_prob(v[0]))

In [ ]:
from edward.models import Dirichlet, Beta, Multinomial, Bernoulli
K=100
N=50

gamma = Dirichlet(concentration=tf.ones([K]))
Pi = Beta(concentration0=tf.ones([K, K]), concentration1=tf.ones([K, K]))
Z = Multinomial(total_count=1.0, probs=gamma, sample_shape=N)
X = Bernoulli(probs=tf.matmul(Z, tf.matmul(Pi, tf.transpose(Z))))

gamma.event_shape

In [ ]:
x.parameters

In [ ]:
x = ed.models.Normal(loc=(0.0,1.0,6.0), scale=(.01,.01,.01))
sess = tf.InteractiveSession()
sess.run()

In [ ]:
x._log_prob(1)

In [ ]:
alpha_test = np.random.uniform(0,1,[5,5])

In [ ]:
alpha_test

In [ ]:
c = Cascade(alpha_test, seed_set=np.array([[1,0,0,1,0],[1,0,0,0,0]]))
sess = tf.InteractiveSession()
sess.run(c._n)

In [ ]:
c.parameters

In [ ]:
# build a custom variable with shape (4 Gamma distributions who are independent)

# MAP using BFGS 

### Implement the model in tensorflow

In [ ]:
# simulate dataset
X_train = np.random.binomial(1, 0.01,size=[N,numNodes])

value = X_train[0].astype(np.float64)

numNodes = 5
alpha_tensor = tf.Variable(tf.zeros([numNodes,numNodes],dtype=tf.float64), dtype=tf.float64)
value = tf.placeholder(tf.float64, [numNodes, 1])    
I = tf.convert_to_tensor(infectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)
U = tf.convert_to_tensor(uninfectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)


psi_1 = -tf.reduce_sum(tf.multiply(alpha_tensor,I))
psi_2 = -tf.reduce_sum(tf.multiply(tf.transpose(alpha_tensor),U))
        
zero_pos = tf.add(tf.multiply(tf.sign(I),-1),tf.ones((numNodes,numNodes),dtype=tf.float64))
psi_3 = tf.reduce_sum(tf.log(tf.add(zero_pos,tf.multiply(tf.transpose(alpha_tensor),tf.sign(I)))))

log_prob = psi_1 + psi_2 + psi_3

In [296]:
## Initialize the values 


# Cascades should be of size c x n; Initialize the variables

C = tf.placeholder(tf.float32, [c,N])

B = tf.Variable(tf.random_normal([N,N]), dtype=tf.float32)

A = tf.nn.softmax(B)
#A = tf.Variable(tf.ones([N,N], dtype=tf.float32), dtype=tf.float32)


I_inf = tf.expand_dims(tf.where(C>0, x=tf.ones((c,N)), y=tf.zeros((c,N))),1)
I_uninf = 1 - tf.expand_dims(tf.where(C>0, x=tf.ones((c,N)), y=tf.zeros((c,N))),1)
t_max = tf.expand_dims(tf.reduce_max(C,1),1)

#calculate dist1
t_mi = tf.expand_dims(tf.subtract(t_max,C),1)

#calculate dist2
k = tf.concat([-tf.ones((c,N,1)),tf.expand_dims(C,-1)], 2)
k_T = tf.transpose(tf.concat([tf.expand_dims(C,-1),tf.ones((c,N,1))], 2),perm=[0,2,1])
t_ij = tf.transpose(tf.matmul(k,k_T),perm=[0,2,1])

# calculate psi_1 

# fix time here!!!!
psi_1 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_uninf, perm=[0,2,1]))
psi_1 = tf.reduce_sum(-tf.multiply(psi_1,t_mi))

#psi_1 = tf.reduce_sum(tf.where(tf.is_inf(psi_1), tf.zeros_like(psi_1), psi_1))


# calculate psi_2

psi_2 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_inf, perm=[0,2,1]))
psi_2 = -tf.multiply(psi_2,t_ij)
psi_2 = tf.reduce_sum(tf.where(t_ij > 0, psi_2,tf.zeros((c,N,N))))



# calculate psi_3

psi_3 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_inf, perm=[0,2,1]))
psi_3 = tf.reduce_sum(tf.where(t_ij > 0, psi_3,tf.zeros((c,N,N))), axis=1)
psi_3 = tf.reduce_sum(tf.where(tf.is_inf(psi_3), tf.zeros_like(psi_3), psi_3))

#psi_3 = tf.multiply(tf.where(t_dist2 > 0, tf.reshape(tf.tile(A,(c,1)),[c,N,N]), tf.zeros((c,N,N))), I_inf)
#psi_3 = tf.transpose(tf.reduce_sum(psi_3,axis=2,keepdims=True), perm=[0,2,1])
#psi_3 = tf.where(tf.logical_not(tf.is_inf(psi_3)), psi_3 , tf.zeros((c,1,N)))
#psi_3 = tf.reduce_sum(-psi_3)

# log(p)

log_p = -(psi_1 + psi_2 + psi_3)
#c_i = tf.exp(log_p)






In [297]:
d = np.array([[0,1,2],
            [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2],
             [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2],
             [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2],
           [0,1,2]])

data = {C: d}
c = d.shape[0]
N = d.shape[1]
max_iter = 1000

optimizer = tf.contrib.opt.ScipyOptimizerInterface(log_p,
                                                   method='L-BFGS-B',
                                                   options={
                                                    'maxiter': max_iter})
model = tf.global_variables_initializer()
sess = tf.Session()
with sess:
    sess.run(model)
    optimizer.minimize(sess, feed_dict=data)
    print(B.eval(session=sess))
    print(A.eval(session=sess))
    

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.000008
  Number of iterations: 15
  Number of functions evaluations: 17
[[ 7.6229234  -7.078423    0.7005998 ]
 [ 0.42220014 -1.4814656   1.120647  ]
 [-0.24601772  1.1918441   1.2852278 ]]
[[9.9901509e-01 4.1196321e-07 9.8456652e-04]
 [3.1649086e-01 4.7163896e-02 6.3634521e-01]
 [1.0167136e-01 4.2820728e-01 4.7012144e-01]]


### Generate a dataset 

TensorShape(None)